In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
import math
from imutils import face_utils
from head_pose import get_points_from_landmarks
from head_pose import HeadPoseEstimator

In [6]:
def load_models():
	facial_features_model = '/home/ubuntu/sourav_kamboj/jupyter_notebook_files/video_analysis/shape_predictor_68_face_landmarks.dat'
	detector = dlib.get_frontal_face_detector()
	predictor = dlib.shape_predictor(facial_features_model)
	return detector, predictor

detector, predictor = load_models()

In [7]:
def get_landmarks(frame, detector=detector, predictor=predictor):
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	rects = detector(gray, 0)

	if str(rects) == 'rectangles[]':
		return None

	for (i, rect) in enumerate(rects):
		landmarks = predictor(gray, rect)
		landmarks = face_utils.shape_to_np(landmarks)
	return landmarks

In [15]:
def orientation_threshold(pitch, yaw):
	direction = []

	if pitch == 0 and yaw == 0:
	    looking = ['Straight']
	    direction.append(looking)
	    
	    
	## Pitch - Up & Down
	if pitch > 0 and pitch < 20:
	    looking = ['Normal Up']
	    direction.append(looking)
	    
	elif pitch > 21 and pitch < 40:
	    looking = ['Moderate Up']
	    direction.append(looking)
	    
	elif pitch > 41:
	    looking = ['Extreme Up']
	    direction.append(looking)
	    
	    
	elif pitch > -20 and pitch < 0:
	    looking = ['Normal Down']
	    direction.append(looking)
	    
	elif pitch > -40 and pitch < -21:
	    looking = ['Moderate Down']
	    direction.append(looking)
	    
	elif pitch < -41:
	    looking = ['Extreme Down']
	    direction.append(looking)
	    

	## Yaw - Left & Right
	if yaw > 0 and yaw < 20:
	    looking = ['Normal Right']
	    direction.append(looking)

	elif yaw > 21 and yaw < 40:
	    looking = ['Moderate Right']
	    direction.append(looking)
	    
	elif yaw > 41:
	    looking = ['Extreme Right']
	    

	elif yaw < 0 and yaw > -40:
	    looking = ['Normal Left']
	    direction.append(looking)

	elif yaw < -21 and yaw > -40:
	    looking = ['Moderate Left']
	    direction.append(looking)
	    
	elif yaw < -41:
	    looking = ['Extreme Left']
	    direction.append(looking)
	    
	return direction

In [16]:
def get_orientation(image_path, draw_direction=False, draw_values=False, mode='nose_chin_eyes_mouth'):

    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255), (255, 255, 0), (255, 125, 125)]
    
    image=cv2.imread(image_path)
    height, width = image.shape[:2]

    ## Model points
    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                           (-165.0, 170.0, -135.0),     # Left eye left corner
                            (165.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                         
                        ])

    ## Center of image
    center = (image.shape[1]/2, image.shape[0]/2)

    ## Focal Length (approx.)
    focal_length = center[0] / np.tan(60/2 * np.pi / 180)

    ## Camera matrix
    camera_matrix = np.array([[focal_length, 0, center[0]],
							 [0, focal_length, center[1]],
							 [0, 0, 1]], dtype = "double")


    ## Distortion Coefficients - Zero for us since assuming no distortion
    
    dist_coeffs = np.zeros((4, 1))    


    ## Getting face landmarks 
    landmarks = get_landmarks(image)

    if landmarks is None:
        return [(0, 0, 0)], image


    ## Pose estimator
    pose_estimator = HeadPoseEstimator(image_size=(height, width), mode=mode)
    image_points = get_points_from_landmarks(landmarks, mode)
    rotation_vector, translation_vector = pose_estimator.solve_pose(image_points)
        
    axis = np.float32([[350, 0, 0], 
	                  [0, 350, 0], 
	                  [0, 0, 350]])






    imgpts, jac = cv2.projectPoints(axis, 
	                                rotation_vector, 
	                                translation_vector, 
	                                camera_matrix, 
	                                dist_coeffs)

    modelpts, jac2 = cv2.projectPoints(model_points, 
	                                   rotation_vector, 
	                                   translation_vector, 
	                                   camera_matrix, 
	                                   dist_coeffs)




	## Convert rotation vector to rotation matrix
    rvec_matrix = cv2.Rodrigues(rotation_vector)[0]   


	## Projection Matrix
    proj_matrix = np.hstack((rvec_matrix, translation_vector))

	## Getting Euler's angles (Roll, Pitch & Yaw)
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6]



	## Pitch, Yaw & Roll
    pitch, yaw, roll = [math.radians(_) for _ in eulerAngles]

	## To degrees
    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))

    face_direction = orientation_threshold(pitch, yaw)


    rotate_degree = (str(int(roll)), str(int(pitch)), str(int(yaw)))

    nose_x = int(image_points[0][0])
    nose_y = int(image_points[0][1])    


	## Drawing lines
	## Conversion
	## X=R, Y=G, Z=B
    cv2.line(image, (nose_x, nose_y), tuple(imgpts[1].ravel()), (0, 255, 0), 3) #GREEN - Y-axis - PITCH
    cv2.line(image, (nose_x, nose_y), tuple(imgpts[2].ravel()), (255, 0, 0), 3) #BLUE - Z-axis - YAW
    cv2.line(image, (nose_x, nose_y), tuple(imgpts[0].ravel()), (0, 0, 255), 3) #RED X-axis - ROLL    


	## Drawing feature points
    for i, pnt in enumerate(image_points.tolist()):
        cv2.circle(image, (int(pnt[0]), int(pnt[1])), 1, colors[i % 6], 3, cv2.LINE_AA)


	# Roll, Pitch & Yaw on image
    if draw_values:
        for j in range(len(rotate_degree)):
            cv2.putText(image, ('{:03.2f}').format(float(rotate_degree[j])), 
                        (10, 30 + (30 * j)), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 0, 0), thickness=1, lineType=1)
            
	        

	## Drawing direction instead of Roll, pitch, yaw values
    if draw_direction:
        for j in range(len(face_direction)):
            cv2.putText(image, '{}'.format(str(face_direction[j])), 
                          (10, 30 + (30 * j)), cv2.FONT_HERSHEY_SIMPLEX, 
                          1, (255, 255, 255), thickness=2, lineType=1)



    return [rotate_degree], image


In [17]:
a,img=get_orientation('./test_images/mark.jpg')

In [19]:
cv2.imwrite('test.jpg',img)

True

In [ ]:
 # Read Image
im = cv2.imread('./test_images/mark.jpg')
size = im.shape
     
#2D image points. If you change the image, you need to change vector
image_points = np.array([
                            (359, 391),     # Nose tip
                            (399, 561),     # Chin
                            (337, 297),     # Left eye left corner
                            (513, 301),     # Right eye right corne
                            (345, 465),     # Left Mouth corner
                            (453, 469)      # Right mouth corner
                        ], dtype="double")
 
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])
 
 
# Camera internals
 
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
 
print("Camera Matrix :\n {0}".format(camera_matrix))
 
dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
(success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.cv2.SOLVEPNP_ITERATIVE)
 
print("Rotation Vector:\n {0}".format(rotation_vector))
print("Translation Vector:\n {0}".format(translation_vector)) 
 
 
# Project a 3D point (0, 0, 1000.0) onto the image plane.
# We use this to draw a line sticking out of the nose
 
#
(nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
 
for p in image_points:
    cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
 
 
p1 = ( int(image_points[0][0]), int(image_points[0][1]))
p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
 
cv2.line(im, p1, p2, (255,0,0), 2)
 
# Display image
#plt.imshow(im)
cv2.imwrite('pose_estimation1.png',im)
#cv2.waitKey(0)